In [1]:
import numpy as np
import matplotlib.pylab as plt
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam

%matplotlib inline
np.set_printoptions(suppress=True)

Using TensorFlow backend.


# SINGLE ITEM

## data

In [2]:
# number of obs
n = 1000

In [3]:
# random prices and other variables
prices = np.random.uniform(20, 200, size=(n, 1)).astype(np.float32)
xs = np.random.rand(n,5).astype(np.float32)

In [4]:
# create beta coefficients
beta_0 = np.array(([300],))
beta_p = np.array(([-1.5],))
betas_x = np.random.normal(size=(5,1)).astype(np.float32)

TODO: markdown for optimal price calculation

In [5]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [6]:
# create demand to forecast
y = beta_0 + beta_p*prices + np.dot(xs, betas_x) + noise
y = np.maximum(0, y)

In [7]:
# train/test ids
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

In [8]:
# optimal price
opt_price = -beta_0/(2*beta_p)
opt_price

array([[ 100.]])

## model

In [9]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [10]:
# concat the inputs
x = Concatenate(name='concat')([inp_p, inp_x])

# single dense hidden layer
x = Dense(3, name='hid')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [11]:
# train
hist = model.fit([prices[trn], xs[trn]], y[trn],
                 validation_data=[[prices[tst], xs[tst]], y[tst]],
                 verbose=0,
                 epochs=200)

## optimization

In [12]:
# make model copy for price
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [13]:
# "freeze" the model
# we want our model for demand to stay the same
# while the price input is optimized
for layer in model_p.layers:
    layer.trainable = False

In [14]:
# dict of layers
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x1182b9748>,
 'hid': <keras.layers.core.Dense at 0x1182b9908>,
 'inp_p': <keras.engine.topology.InputLayer at 0x1182b9438>,
 'inp_x': <keras.engine.topology.InputLayer at 0x1182b9630>,
 'out': <keras.layers.core.Dense at 0x12148bdd8>}

In [15]:
# add a single unit dense inbetween price input and hidden layer
# this will create a single trainable weight
# we will feed the input all 1's so it can learn the optimal price
p = Dense(1, use_bias=False, name='price')(ls['inp_p'].output)

In [16]:
# concat the inputs and send through hidden layers like before
x = ls['concat']([p, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

In [17]:
# our output of the last model was an estimate for demand
# so now we multiply that by the trained price to get revenue
out = Multiply(name='revenue_out')([p, x])

In [18]:
# build model
model_p = Model([ls['inp_p'].input, ls['inp_x'].input], out)

In [19]:
# we want to maximize revenue, but keras minizises loss
# so we'll just make our loss the negative revenue
def rev_loss(y_true, y_pred):
    return -K.mean(y_pred)

In [20]:
# compile model
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [21]:
# callback to track layer weights
class WtTracker(keras.callbacks.Callback):
    def __init__(self, layer, display=5):
        self.seen = 0
        self.layer = layer
        self.display = display

    def on_epoch_end(self,batch,logs={}):
        self.seen += 1
        if self.seen % self.display == 0:
            wts = self.layer.get_weights()[0].flatten().round(2)
            print(f'epoch {self.seen}, price {wts}')

In [22]:
# train
hist = model_p.fit([np.ones((len(xs), 1)), xs], y,
                   epochs=120,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('price'))])

epoch 5, price [ 16.57999992]
epoch 10, price [ 30.71999931]
epoch 15, price [ 43.47999954]
epoch 20, price [ 54.81000137]
epoch 25, price [ 64.69000244]
epoch 30, price [ 73.12000275]
epoch 35, price [ 80.12000275]
epoch 40, price [ 85.76999664]
epoch 45, price [ 90.16000366]
epoch 50, price [ 93.45999908]
epoch 55, price [ 95.81999969]
epoch 60, price [ 97.44000244]
epoch 65, price [ 98.5]
epoch 70, price [ 99.15000153]
epoch 75, price [ 99.52999878]
epoch 80, price [ 99.73999786]
epoch 85, price [ 99.83999634]
epoch 90, price [ 99.88999939]
epoch 95, price [ 99.91999817]
epoch 100, price [ 99.93000031]
epoch 105, price [ 99.93000031]
epoch 110, price [ 99.93000031]
epoch 115, price [ 99.93000031]
epoch 120, price [ 99.93000031]


# MULTIPLE ITEMS

## data

In [23]:
n = 5000
prices = np.random.uniform(50, 200, size=(n, 1)).astype(np.float32)
xs = np.random.normal(size=(n,5)).astype(np.float32)

In [24]:
# create input for item ids, we'll make 5
ids = np.tile([0,1,2,3,4], n//5).reshape(-1,1)

In [25]:
# create beta coefficients
beta_0 = np.array([500],)

beta_p = np.array([-1.4,
                   -1.5,
                   -1.6,
                   -1.8,
                   -2.0],)

betas_x = np.random.normal(size=(5,1)).astype(np.float32)

In [26]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [27]:
# create demand to forecast
y = beta_0 + beta_p[ids]*prices + np.dot(xs, betas_x) + noise
y = np.maximum(0, y)

In [28]:
# train/test ids
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

In [29]:
# derived optimal prices
opt_prices = (-beta_0/(2*beta_p)).round(2)
opt_prices

array([ 178.57,  166.67,  156.25,  138.89,  125.  ])

## model

In [30]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [31]:
# use embeddings to learn the "coefficient" for each item
inp_ids = Input((1,), name='inp_ids')
ids_emb = Embedding(input_dim=5, output_dim=1, name='ids_emb')(inp_ids)
ids_emb = Reshape((1,), name='ids_emb_re')(ids_emb)

# multiply the embedding coefficient by price
x = Multiply(name='mult_p_emb')([ids_emb, inp_p])

In [32]:
# concat the inputs
x = Concatenate(name='concat')([x, inp_x])

# single dense hidden layer
x = Dense(10, name='hid')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_ids, inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [33]:
# train
hist = model.fit([ids[trn], prices[trn], xs[trn]], y[trn],
                 validation_data=[[ids[tst], prices[tst], xs[tst]], y[tst]],
                 verbose=0,
                 epochs=50)

## optimization

In [34]:
# make model copy for price
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [35]:
# "freeze" model
for layer in model_p.layers:
    layer.trainable = False

In [36]:
# layers
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x1218d0588>,
 'hid': <keras.layers.core.Dense at 0x121999e48>,
 'ids_emb': <keras.layers.embeddings.Embedding at 0x121577ef0>,
 'ids_emb_re': <keras.layers.core.Reshape at 0x121577cc0>,
 'inp_ids': <keras.engine.topology.InputLayer at 0x121699be0>,
 'inp_p': <keras.engine.topology.InputLayer at 0x121699630>,
 'inp_x': <keras.engine.topology.InputLayer at 0x121699470>,
 'mult_p_emb': <keras.layers.merge.Multiply at 0x12182bb70>,
 'out': <keras.layers.core.Dense at 0x1219edac8>}

In [37]:
# id price embeddings
# lets us "train" a price for every id
p = Embedding(input_dim=5, output_dim=1, name='ids_price')(ls['inp_ids'].output)
p = Reshape((1,), name='ids_price_re')(p)

# multiply price, concat, and send through desnse layers like before
x = ls['mult_p_emb']([p, ls['ids_emb_re'].output, ls['inp_p'].output])
x = ls['concat']([x, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

# revenue output
out = Multiply(name='revenue_out')([p, x])

In [38]:
# build/compile model
model_p = Model([ls['inp_ids'].input, ls['inp_p'].input, ls['inp_x'].input], out)
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [39]:
# display optimal prices for reference
opt_prices

array([ 178.57,  166.67,  156.25,  138.89,  125.  ])

In [40]:
# train
hist = model_p.fit([ids, np.ones((len(xs), 1)), xs], y,
                   epochs=50,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('ids_price'))])

epoch 5, price [ 67.01000214  66.05000305  65.19999695  64.58000183  63.43999863]
epoch 10, price [ 117.98999786  115.51000214  112.68000031  108.44000244  103.01000214]
epoch 15, price [ 152.3999939   147.27000427  141.41999817  131.6000061   120.13999939]
epoch 20, price [ 170.41999817  162.21000671  153.61000061  139.16000366  124.05999756]
epoch 25, price [ 176.41999817  166.3999939   156.41999817  140.32000732  124.41000366]
epoch 30, price [ 177.41000366  166.91999817  156.67999268  140.36999512  124.41000366]
epoch 35, price [ 177.46000671  166.94000244  156.69000244  140.38000488  124.41000366]
epoch 40, price [ 177.47000122  166.94999695  156.69000244  140.36999512  124.41999817]
epoch 45, price [ 177.46000671  166.94999695  156.69000244  140.38999939  124.41999817]
epoch 50, price [ 177.46000671  166.96000671  156.66999817  140.36999512  124.45999908]


In [41]:
# percents off from optimal prices
(model_p.get_layer('ids_price').get_weights()[0].flatten() / opt_prices - 1).round(3)

array([-0.006,  0.002,  0.003,  0.011, -0.004])

# SINGLE ITEM - QUADRATIC

## data

In [42]:
# number of obs
n = 1000

In [43]:
# random prices and other variables
prices = np.random.uniform(20, 60, size=(n, 1)).astype(np.float32)
xs = np.random.rand(n,5).astype(np.float32)

In [44]:
# create beta coefficients
beta_0 = np.array(([400],))
beta_p = np.array(([1.0],))
beta_p2 = np.array(([-0.1],))
betas_x = np.random.normal(size=(5,1)).astype(np.float32)

In [45]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [46]:
# create demand to forecast
y = beta_0 + beta_p*prices + beta_p2*(prices**2) + np.dot(xs, betas_x) + noise

In [47]:
# plt.hist(y); plt.show()

In [48]:
# train/test ids
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

In [49]:
# optimal price
# a*x**2 + b*x + c = 0
c,b,a = beta_0, 2*beta_p, 3*beta_p2
d = b**2-4*a*c # discriminant

if d < 0:
    print ('No solutions')
elif d == 0:
    x1 = -b / (2*a)
    print ('The sole solution is',x1)
else: # if d > 0
    x1 = (-b + np.sqrt(d)) / (2*a)
    x2 = (-b - np.sqrt(d)) / (2*a)
    print ('Solutions are',x1,'and',x2)
    
opt_price = max(x1,x2)

Solutions are [[-33.33333333]] and [[ 40.]]


## model

In [50]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [51]:
# concat the inputs
x = Concatenate(name='concat')([inp_p, inp_x])

# single dense hidden layer
x = Dense(20, name='hid', activation='relu')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [52]:
# train
hist = model.fit([prices[trn], xs[trn]], y[trn],
                 validation_data=[[prices[tst], xs[tst]], y[tst]],
                 verbose=0,
                 epochs=200)

## optimization

In [53]:
# make model copy
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [54]:
# freeze model
for layer in model_p.layers:
    layer.trainable = False

In [55]:
# layers
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x121cf2780>,
 'hid': <keras.layers.core.Dense at 0x121ce4cc0>,
 'inp_p': <keras.engine.topology.InputLayer at 0x121cf2940>,
 'inp_x': <keras.engine.topology.InputLayer at 0x121cf2400>,
 'out': <keras.layers.core.Dense at 0x121fcceb8>}

In [56]:
# add layer for price input
p = Dense(1, use_bias=False, name='price')(ls['inp_p'].output)

# concat the inputs and send through hidden layers like before
x = ls['concat']([p, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

# revenue output
out = Multiply(name='revenue_out')([p, x])

In [57]:
# build/compile model
model_p = Model([ls['inp_p'].input, ls['inp_x'].input], out)
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [58]:
# display optimal price
opt_price

array([[ 40.]])

In [59]:
# train
hist = model_p.fit([np.ones((len(xs), 1)), xs], y,
                   epochs=100,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('price'))])

epoch 5, price [ 14.06000042]
epoch 10, price [ 24.95000076]
epoch 15, price [ 32.08000183]
epoch 20, price [ 36.04999924]
epoch 25, price [ 37.88999939]
epoch 30, price [ 38.59999847]
epoch 35, price [ 38.83000183]
epoch 40, price [ 38.88999939]
epoch 45, price [ 38.90000153]
epoch 50, price [ 38.90000153]
epoch 55, price [ 38.90000153]
epoch 60, price [ 38.90000153]
epoch 65, price [ 38.90000153]
epoch 70, price [ 38.90000153]
epoch 75, price [ 38.90000153]
epoch 80, price [ 38.90000153]
epoch 85, price [ 38.90000153]
epoch 90, price [ 38.90000153]
epoch 95, price [ 38.90000153]
epoch 100, price [ 38.90000153]
